In [74]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io.fits import getdata

import pandas as pd
numaster = getdata('numaster.fits')

In [75]:
df = pd.read_csv('stray_light_det.txt', sep='\s+', names=['SEQID', 'FPM', 'SRC', 'DET0', 'DET1', 'DET2', 'DET3'])

# Filter out cases where the source is really bright:
bright_lim = 100.
df = df[df['SRC']<100.]


df['DET0123'] = df['DET0'] + df['DET1'] + df['DET2'] + df['DET3']
df['DET01'] = df['DET0'] + df['DET1']
df['DET12'] = df['DET1'] + df['DET2']
df['DET23'] = df['DET2'] + df['DET3']
df['DET03'] = df['DET0'] + df['DET3']


names = []
exp = []
ra = []
dec = []
pa = []
for row in df['SEQID']:
    nuind = np.where(numaster['OBSID'] == f'{row}')
    if (len(nuind[0]) == 0):
        names = np.append(names, '')
        exp = np.append(exp, 0)
        ra = np.append(ra, 0.)
        dec = np.append(dec, 0.)
        pa = np.append(pa, 0.)
    else:
        names = np.append(names, numaster['NAME'][nuind[0]].strip())
        exp = np.append(exp, numaster['EXPOSURE_A'][nuind[0]])
        ra = np.append(ra,numaster['RA'][nuind[0]] )
        dec = np.append(dec,numaster['DEC'][nuind[0]] )
        pa = np.append(pa,numaster['ROLL_ANGLE'][nuind[0]] )
        
df['NAME'] = names
df['EXPOSURE'] = exp
df['RA'] = ra
df['DEC'] = dec
df['PA'] = pa

# Get rid of stuff with low exposure
df = df[df['EXPOSURE']>1e3]

# Get rid of stuff near SgrA*, GalSurveys, Gal_Cen, or the GC Magnetar:
# Also Coma and Bullet cluster stuff
bad_name = ['Sgr', 'GalSurvey', 'Gal_Cen', 'gcmag', 'Coma', 'bullet', 'Bullet']
for bn in bad_name:   
    df = df[~(df['NAME'].str.startswith(bn))]

In [76]:
sig_list = []

with open('MergedAB.csv', 'r') as f:
    for line in f:
        fields = line.split(',')
#         if fields[0].strip() == '':
#             break
        if fields[1] not in sig_list:
            sig_list = np.append(sig_list, fields[1])
sig_list = sig_list[1:]

In [77]:
print(len(df))
print(len(sig_list))

for seq in sig_list:
    try:
        seqid = float(seq)
    except:
        continue
    df = df[~(df['SEQID'] == seqid)].copy().reset_index(drop=True)
print(len(df))


5530
545
4510


In [79]:
# Filter out bright sources and short exposures
df = df[(df['SRC'] < 2.) & (df['EXPOSURE'] > 10e3)].copy().reset_index(drop=True)
print(len(df))

3840


In [85]:
dfa = df[df['FPM']=='A'].copy().reset_index(drop=True)
dfb = df[df['FPM']=='B'].copy().reset_index(drop=True)

running_list = {'A':[], 'B':[]}


# Iteration step: find stuff 3-sigma

sig_limit = 3.0
for det_comb in ['DET0123', 'DET12', 'DET23', 'DET01', 'DET03', 'DET1', 'DET2', 'DET3', 'DET0']:
    
    
    use = {'A':np.full(len(dfa), True, dtype=bool),
           'B':np.full(len(dfb), True, dtype=bool)}
    for fpm, dfi in zip(['A', 'B'], [dfa, dfb]):
        with open(f'sig_cuts_{det_comb}{fpm}_lvl2.txt', 'w') as f:
            for iter in range(3):
                dfi['SIG'] = (dfi[det_comb] - dfi[det_comb].mean()) / dfi[det_comb].std()
                high_sig = dfi[dfi['SIG'] > sig_limit]

                for ind, row in high_sig.iterrows():
                    use[fpm][ind] = False
                    if row['SEQID'] not in running_list[fpm]:
                        running_list[fpm] = np.append(running_list[fpm], row['SEQID'])
                        outstring = f"{row['SEQID']}, {row['NAME']}, {row['EXPOSURE']}, {row['RA']}, {row['DEC']}, {row['PA']}, {iter}"
                        print(fpm, row['SRC'], row['EXPOSURE'], row['SIG'], outstring)
                        f.write(outstring+'\n')
                    dfi = dfi[dfi['SIG']<sig_limit].copy()

    # Now get rid of everything that you've already used for the next round of searching.
    dfa = dfa[use['A']].copy().reset_index(drop=True)
    dfb = dfb[use['B']].copy().reset_index(drop=True)

A 0.17244794380196107 132005.0532 3.0390247537671455 40401003002, 3C_397, 132005.0532, 286.8695, 7.2104, 119.2559, 0
A 0.14242104748052936 25831.8561 3.4307633990762842 60361015002, MRK0315, 25831.8561, 346.0374, 22.6588, 145.1307, 0
A 1.572357770770603 58351.2237 4.118414303777354 30361002004, SXP_15d3, 58351.2237, 13.1412, -73.3495, 263.2395, 0
A 0.4356847453602922 46877.9323 7.426264363909672 90201021010, Kepler, 46877.9323, 262.6131, -21.5578, 6.2795, 0
A 0.11266112431916528 37359.826 3.142887980815534 90401312002, IGR_J17379m3747, 37359.826, 264.5251, -37.7505, 161.1978, 0
A 0.471033128821796 26509.8127 4.909942087638375 30364003002, PSR_J1813m1749, 26509.8127, 273.4203, -17.8097, 152.9763, 0
A 0.21255830060728007 40003.1426 6.21563176334461 90302321004, PSR_J2032p4127, 40003.1426, 308.0475, 41.4336, 306.9602, 0
A 0.7411110376406935 95271.8235 3.2244389509605003 40301004002, KES_75, 95271.8235, 281.5856, -3.0002, 335.5333, 0
A 0.16800213467426325 74802.621 3.2536596048839734 30302

In [86]:
print(len(running_list['A']))
print(len(running_list['B']))



389
369
